In [ ]:
### Import pandas liabrary and read the wiki page into a pandas dataframe ###

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
canada = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',)

In [ ]:
### assign the column names to the dataframe and drop unneeded row

In [2]:
canada_post = pd.DataFrame(canada[0])
canada_post.columns = ['Postcode', 'Borough', 'Neighbourhood']
canada_post.drop(0, axis = 0, inplace = True)
canada_post.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [ ]:
### create a mask to filter out any row whose 'Borough' column is 'Not assigned'

In [3]:
mask = canada_post['Borough'] != 'Not assigned'
canada_post_1 = canada_post.loc[mask,:]
canada_post_1.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [ ]:
### find the row with 'Neighbourhood' is 'Not assigned'

In [4]:
canada_post_1[canada_post_1['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
9,M7A,Queen's Park,Not assigned


In [ ]:
### Replace the 'Not assigned' with 'Borough' value and re-check

In [5]:
canada_post_1.loc[9,'Neighbourhood'] = "Queen's Park"
canada_post_1[canada_post_1['Neighbourhood'] == 'Not assigned']

D:\Work\Anaconda\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
D:\Work\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood


In [ ]:
### check the data structure of the group object ###

In [6]:
canada_post_group = canada_post_1.groupby(['Postcode','Borough'])['Neighbourhood']
pd.Series(canada_post_group)[:5]

0               ((M1B, Scarborough), [Rouge, Malvern])
1    ((M1C, Scarborough), [Highland Creek, Rouge Hi...
2    ((M1E, Scarborough), [Guildwood, Morningside, ...
3                       ((M1G, Scarborough), [Woburn])
4                    ((M1H, Scarborough), [Cedarbrae])
dtype: object

In [ ]:
### combine 'Neighbourhood' values using ', ', and store the result in a dataframe ###

In [7]:
canada_post_df = pd.DataFrame(canada_post_group.apply(lambda x: ', '.join(x)))
canada_post_df.reset_index(inplace = True)
canada_post_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
canada_post_df.shape

(103, 3)

In [ ]:
### install geocoder if current python doesn't have it ###

In [4]:
!pip install geocoder

  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: C:\Users\paulf\AppData\Local\pip\Cache\wheels\2c\02\af\63eadc269fe686aa0aa9c38eee165ad5734cbf8b765cfeedaa
Successfully built future


You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
### Loop throuhg each row of the dataframe and get corresponding coordinates ###

In [ ]:
import geocoder

for index, data in canada_post_df.iterrows():
    lat_lng_coords = None
    while (lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(data[0]))
        lat_lng_coords = g.latlng
    canada_post_df.loc[index, 'Latitude'] = lat_lng_coords[0]
    canada_post_df.loc[index, 'Longitude'] = lat_lng_coords[1]

canada_post_df.head()


In [ ]:
### Alternative in case that geocoder doesn't work or takes too much time

In [9]:
canada_lat_lng = pd.read_csv('http://cocl.us/Geospatial_data')

In [10]:
canada_lat_lng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
for index, data in canada_lat_lng.iterrows():
    mask = canada_post_df['Postcode'] == data[0]
    canada_post_df.loc[mask, 'Latitude'] = data[1]
    canada_post_df.loc[mask, 'Longitude'] = data[2]
canada_post_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
 #cluster venue_category, find top 10 venues and their category

In [12]:
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
import datetime
from sklearn.cluster import KMeans as km

In [13]:
log_info = pd.read_csv(r'C:\Users\paulf\OneDrive\Desktop\foursquare.csv')
log_info

,client_id,client_secret
0,VQNAGFFMJK5F5LCHVOB4VCQBXEUVI35TGNK5CX2EH4O5MHZV,YBNGL5OCCF2PMYXVMMATZOAZVABZSVG5GUFH0KA0RV50QXRX


In [14]:
version = datetime.date.today().strftime('%Y%m%d')
version

'20191026'

In [ ]:
### using foursquare API to get venues information ####
### reading json files to extract venues information ###
### Extract the names, location['lat'] and location['lng'], categories[0]['name'] ###
### Combine venues information with postcode information ###
### Create top 10 Venues Dataframe ###
### Create dummnies for venue categories, which are used in K-Means clustering ###

In [15]:
venue_header = ['venue_name', 'venue_lat', 'venue_lng', 'venue_cat', 'Postcode']

In [16]:
def get_venue_info(names, lats, lngs, radius = 1000, limit = 100):
    venue_col = []
    error_col = []
    for lat, lng, name in zip(lats, lngs, names):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(log_info.iloc[0,0], 
                                                                                                                                log_info.iloc[0,1],
                                                                                                                                version,
                                                                                                                                lat,lng, radius,limit)
        result = requests.get(url).json()
        try: 
            result = result['response']['groups'][0]['items']
            for element in result:
                venue_col.append([element['venue']['name'],element['venue']['location']['lat'], 
                                  element['venue']['location']['lng'], element['venue']['categories'][0]['name'], name])
        except:
            error_col.append((lat, lng, name))
    nearby_venues = pd.DataFrame(venue_col, columns = venue_header)
    return (nearby_venues, error_col)
    
 

In [17]:
temp = get_venue_info(canada_post_df['Postcode'], canada_post_df['Latitude'], canada_post_df['Longitude'], 500)
toronto_venues = temp[0]

In [18]:
toronto_venues.head()

,venue_name,venue_lat,venue_lng,venue_cat,Postcode
0,Wendy's,43.807448,-79.199056,Fast Food Restaurant,M1B
1,Royal Canadian Legion,43.782533,-79.163085,Bar,M1C
2,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place,M1E
3,G & G Electronics,43.765309,-79.191537,Electronics Store,M1E
4,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant,M1E


In [19]:
toronto = canada_post_df.merge(toronto_venues, how = 'outer', left_on = 'Postcode', right_on = 'Postcode')
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,venue_name,venue_lat,venue_lng,venue_cat
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [ ]:
### Analyze the areas that have 10 or more venues ###

In [20]:
toronto_group = toronto.groupby(['Postcode']).count()
temp_list = toronto_group[toronto_group['Borough'] >= 10].index.tolist()
temp_list[:5]

['M1T', 'M1W', 'M2J', 'M2N', 'M3C']

In [21]:
mask = toronto.Postcode.isin(temp_list)
toronto = toronto.loc[mask,:]
one_hot = pd.get_dummies(toronto['venue_cat'])
one_hot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
toronto = toronto.merge(one_hot, left_index = True, right_index = True)
toronto.reset_index(drop = True)
toronto_group = toronto.groupby('Postcode').mean()

In [23]:
toronto_group.drop(columns = ['venue_lat', 'venue_lng'], inplace = True)
toronto_group.head()

,Latitude,Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Postcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
M1T,43.781637,-79.304302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.

In [ ]:
### search the best parameters for k-means ###

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
cluster_rng = list(range(5,10))
init_rng = list(range(10,15))
iter_rng = [300,400,500,600,700]
param_dict ={'n_clusters': cluster_rng, 'n_init': init_rng, 'max_iter': iter_rng}
X = toronto_group.iloc[:, 2:]

gridsearch = GridSearchCV(km(), param_grid = param_dict)
gridsearch.fit(X)

D:\Work\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
D:\Work\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KMeans(algorithm='auto', copy_x=True, init='k-means++',
                              max_iter=300, n_clusters=8, n_init=10,
                              n_jobs=None, precompute_distances='auto',
                              random_state=None, tol=0.0001, verbose=0),
             iid='warn', n_jobs=None,
             param_grid={'max_iter': [300, 400, 500, 600, 700],
                         'n_clusters': [5, 6, 7, 8, 9],
                         'n_init': [10, 11, 12, 13, 14]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [26]:
gridsearch.best_params_

{'max_iter': 400, 'n_clusters': 5, 'n_init': 10}

In [27]:
kmeans = km(n_init = 10, n_clusters = 5, max_iter = 400, random_state = 0)
label = kmeans.fit(X).labels_
label

array([0, 0, 4, 4, 4, 1, 0, 4, 4, 1, 4, 1, 1, 4, 4, 4, 4, 1, 4, 4, 4, 1,
       4, 1, 1, 1, 4, 1, 4, 4, 4, 1, 1, 2, 3, 4, 4, 4, 4, 4, 4, 1, 1, 4,
       4, 4, 0])

In [28]:
label = pd.DataFrame(label, columns = ['Label'])
toronto_group.reset_index(inplace = True)
toronto_group.head()

,Postcode,Latitude,Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1T,43.781637,-79.304302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.100000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.100000,0.0,0.0,0.0,0.000000,0.0,0.00

In [29]:
toronto_group = pd.merge(label, toronto_group, how = 'inner', left_index = True, right_index = True)

In [30]:
toronto_group.sort_values('Label', axis = 0, inplace = True)

In [ ]:
### import Folium and create a map ###

In [32]:
import folium
import geopy
from geopy.geocoders import Nominatim

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent = 'google')
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lng = location.longitude
print ("Toronto's geocode is {}, {}".format(toronto_lat, toronto_lng))

Toronto's geocode is 43.653963, -79.387207


In [33]:
from matplotlib import cm
from matplotlib import colors
x = np.arange(5)
ys = [i+x+(i*x)**2 for i in range(5)]
color_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in color_array]


In [34]:
map_cluster = folium.Map(location = [toronto_lat, toronto_lng], zoom_start = 11)

for lat, lng, post, label in zip(toronto_group.Latitude, toronto_group.Longitude, toronto_group.Postcode, toronto_group.Label):
    pop_label = folium.Popup(post + '_' + str(label), parse_html = True)
    folium.CircleMarker(
        [lat, lng], radius = 5, popup = pop_label,
        color = rainbow[label], fill = True, fill_color = rainbow[label],
        fill_opacity = 0.7).add_to(map_cluster)
    
map_cluster